In [2]:
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import time
from bs4 import BeautifulSoup

with open('Data/general_list.json', 'r') as file:
    data = json.load(file)
general_names = [sublist[0] for sublist in data]
print(len(general_names))


86582


In [5]:
base_url = 'https://anw.ivdnt.org/article/'
words = []
xml_files = []
max_retries = 3
retry_delay = 5 
progress_bar = tqdm(general_names, desc='Processing')
for word in progress_bar:
    retries = 0
    while retries < max_retries:
        url = base_url + word + '?output=xml'
        try:
            response = requests.get(url)
            if response.status_code == 200:
                xml_content = response.content
                words.append(word)
                xml_files.append(xml_content)
                break 
        except requests.exceptions.RequestException as e:
            print(f"Attempt {retries+1} failed:", e)
        
        retries += 1
        time.sleep(retry_delay)
df = pd.DataFrame({'word': words, 'xml_file': xml_files})

Processing:  63%|██████▎   | 54191/86582 [3:27:22<1:57:54,  4.58it/s] 

Attempt 1 failed: HTTPSConnectionPool(host='anw.ivdnt.org', port=443): Max retries exceeded with url: /article/pati%C3%ABntiewerk?output=xml (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7fbf246ee1b0>: Failed to resolve 'anw.ivdnt.org' ([Errno -3] Temporary failure in name resolution)"))


Processing:  87%|████████▋ | 75289/86582 [4:49:56<15:05:55,  4.81s/it]

Attempt 2 failed: HTTPSConnectionPool(host='anw.ivdnt.org', port=443): Max retries exceeded with url: /article/verleidelijk?output=xml (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fbf1eeefa70>: Failed to establish a new connection: [Errno 111] Connection refused'))


Processing: 100%|██████████| 86582/86582 [5:33:42<00:00,  4.32it/s]   


In [6]:
df.to_csv('Data/dut_general_xml.csv', index = False)

In [4]:
url = 'https://anw.ivdnt.org/article/achterbalkon?output=xml'
response = requests.get(url)
if response.status_code == 200:
    xml_content = response.content
    print(xml_content)
    

b'<artikel last-modified="2023-11-28" kopje="true" ID="997" pid="2692"> <Lemma> <Lemmavorm>achterbalkon</Lemmavorm> <Lemmatype>woord</Lemmatype> </Lemma>       <BetekenisEnGebruik> <Kernbetekenis pid="2693" id="bet1.0"> <betekenisInfo> <Betekenisnummer>1.0</Betekenisnummer> <Lemma> <Lemmavorm>achterbalkon</Lemmavorm> <Lemmatype>woord</Lemmatype> </Lemma> <SpellingDbId>2066</SpellingDbId> <Woordsoort> <Type>substantief</Type> <Substantief> <Naamtype>soortnaam</Naamtype> <GeslachtBody id="el2"> <Geslacht>onzijdig</Geslacht> </GeslachtBody> <Lidwoord id="el3"> <AardLidwoord>het</AardLidwoord> </Lidwoord> <Betekenisklasse>zaaknaam</Betekenisklasse> </Substantief> </Woordsoort> <SpellingEnFlexie> <SubstantiefVormen> <Enkelvoud LexiconId="157780"> <Woordvorm>achterbalkon</Woordvorm> <Afbreking>ach.ter.bal.kon</Afbreking> </Enkelvoud> <Meervoud LexiconId="207639"> <Woordvorm>achterbalkons</Woordvorm> <Afbreking>ach.ter.bal.kons</Afbreking> </Meervoud> <Verkleiningsvorm LexiconId="804160"> <Wo

In [39]:
df.iloc[0]['xml_file']

b'<artikel last-modified="2023-10-04" ID="151279" pid="493255"> <Lemma> <Lemmavorm>aanbiddingszanger</Lemmavorm> <Lemmatype>woord</Lemmatype> </Lemma>       <BetekenisEnGebruik> <Kernbetekenis neo="true" pid="493257" id="bet1.0"> <betekenisInfo> <Betekenisnummer>1.0</Betekenisnummer> <Lemma> <Lemmavorm>aanbiddingszanger</Lemmavorm> <Lemmatype>woord</Lemmatype> </Lemma> <SpellingDbId>170196</SpellingDbId> <Woordsoort> <Type>substantief</Type> <Substantief> <Naamtype>soortnaam</Naamtype> <GeslachtBody id="el6"> <Geslacht>mannelijk</Geslacht> </GeslachtBody> <Lidwoord id="el7"> <AardLidwoord>de</AardLidwoord> </Lidwoord> <Betekenisklasse>persoonsnaam</Betekenisklasse> </Substantief> </Woordsoort> <SpellingEnFlexie> <SubstantiefVormen> <Enkelvoud LexiconId="502408"> <Woordvorm>aanbiddingszanger</Woordvorm> <Afbreking>aan.bid.dings.zan.ger</Afbreking> </Enkelvoud> <Meervoud LexiconId="486263"> <Woordvorm>aanbiddingszangers</Woordvorm> <Afbreking>aan.bid.dings.zan.gers</Afbreking> </Meervoud

In [40]:
def extract_xml(xml_content):
    soup = BeautifulSoup(xml_content, 'html.parser') 
    return soup.prettify()
progress_bar = tqdm(total=len(df), desc='Processing')
df['parsed_xml'] = df['xml_file'].apply(lambda x: (progress_bar.update(), extract_xml(x)))
progress_bar.close()

Processing:   0%|          | 0/1969 [00:00<?, ?it/s]

Processing: 100%|██████████| 1969/1969 [00:28<00:00, 68.08it/s]


In [41]:
df.head()

,word,xml_file,parsed_xml
0,aanbiddingszanger,"b'<artikel last-modified=""2023-10-04"" ID=""1512...","(None, <artikel id=""151279"" last-modified=""202..."
1,aanblijfpremie,"b'<artikel last-modified=""2023-10-04"" ID=""7307...","(None, <artikel id=""73079"" last-modified=""2023..."
2,aanbodgedreven,"b'<artikel last-modified=""2023-10-04"" ID=""7308...","(None, <artikel id=""73080"" last-modified=""2023..."
3,aanbodgestuurd,"b'<artikel last-modified=""2023-10-04"" ID=""7308...","(None, <artikel id=""73081"" last-modified=""2023..."
4,aandachtseconomie,"b'<artikel last-modified=""2024-02-02"" ID=""1669...","(None, <artikel id=""166967"" last-modified=""202..."


In [42]:
df.parsed_xml[0]

(None,
 '<artikel id="151279" last-modified="2023-10-04" pid="493255">\n <lemma>\n  <lemmavorm>\n   aanbiddingszanger\n  </lemmavorm>\n  <lemmatype>\n   woord\n  </lemmatype>\n </lemma>\n <betekenisengebruik>\n  <kernbetekenis id="bet1.0" neo="true" pid="493257">\n   <betekenisinfo>\n    <betekenisnummer>\n     1.0\n    </betekenisnummer>\n    <lemma>\n     <lemmavorm>\n      aanbiddingszanger\n     </lemmavorm>\n     <lemmatype>\n      woord\n     </lemmatype>\n    </lemma>\n    <spellingdbid>\n     170196\n    </spellingdbid>\n    <woordsoort>\n     <type>\n      substantief\n     </type>\n     <substantief>\n      <naamtype>\n       soortnaam\n      </naamtype>\n      <geslachtbody id="el6">\n       <geslacht>\n        mannelijk\n       </geslacht>\n      </geslachtbody>\n      <lidwoord id="el7">\n       <aardlidwoord>\n        de\n       </aardlidwoord>\n      </lidwoord>\n      <betekenisklasse>\n       persoonsnaam\n      </betekenisklasse>\n     </substantief>\n    </woordsoort

In [43]:
df['parsed_xml'] = df['parsed_xml'].apply(lambda x: str(x[1]))


In [49]:
result = True
for index, row in df.iterrows():
    soup = BeautifulSoup(row['parsed_xml'], 'html.parser')
    lemma_element = soup.find('lemma')
    if not lemma_element:
        result = False
        print(index)
print(result)


True
Lemma element not found.


In [53]:
df.to_csv('Data/dut_neo_xml.csv', index = False)

In [7]:
df.head()

,word,xml_file
0,#MeToo,b'\n\n\n\n\n\n\n\n<!DOCTYPE html>\n<html lang=...
1,007,"b'<artikel last-modified=""2024-01-09"" kopje=""t..."
2,06-dealer,"b'<artikel last-modified=""2024-02-23"" ID=""7121..."
3,06-nummer,"b'<artikel last-modified=""2024-03-01"" ID=""6896..."
4,06-prostitutie,"b'<artikel last-modified=""2023-10-04"" ID=""8749..."


In [9]:
df.iloc[1]['xml_file']

b'<artikel last-modified="2024-01-09" kopje="true" ID="156529" pid="932425"> <Lemma> <Lemmavorm>007</Lemmavorm> <Lemmatype>woord</Lemmatype> </Lemma>      <BetekenisEnGebruik> <Kernbetekenis pid="932426" id="bet1.0"> <betekenisInfo> <Betekenisnummer>1.0</Betekenisnummer> <Lemma> <Lemmavorm>007</Lemmavorm> <Lemmatype>woord</Lemmatype> </Lemma> <SpellingDbId>905043</SpellingDbId> <Woordsoort> <Type>substantief</Type> <Substantief> <Naamtype>soortnaam</Naamtype> <GeslachtBody id="el2"> <Geslacht>mannelijk of vrouwelijk</Geslacht> </GeslachtBody> <Lidwoord id="el3"> <AardLidwoord>de</AardLidwoord> </Lidwoord> <Betekenisklasse>persoonsnaam</Betekenisklasse> </Substantief> </Woordsoort> <SpellingEnFlexie> <SubstantiefVormen> <Enkelvoud LexiconId="715292"> <Woordvorm>007</Woordvorm> <Afbreking>007</Afbreking> </Enkelvoud> <Meervoud LexiconId="715293"> <Woordvorm>007\'s</Woordvorm> <Afbreking>007\'s</Afbreking> </Meervoud> </SubstantiefVormen> </SpellingEnFlexie> <Uitspraak> <AantalLettergrepe

In [16]:
adipositas_entry = df[df['word'] == 'adipositas']
adipositas_entry['xml_file']

2013    b'<artikel last-modified="2023-10-04" ID="1496...
Name: xml_file, dtype: object

In [21]:
print(adipositas_entry)

            word                                           xml_file
2013  adipositas  b'<artikel last-modified="2023-10-04" ID="1496...


In [23]:
df.iloc[2013]['xml_file']

b'<artikel last-modified="2023-10-04" ID="149693" pid="481964"> <Lemma> <Lemmavorm>adipositas</Lemmavorm> <Lemmatype>woord</Lemmatype> </Lemma>       <BetekenisEnGebruik> <Kernbetekenis pid="879179" id="bet1.0"> <betekenisInfo> <Betekenisnummer>1.0</Betekenisnummer> <Lemma> <Lemmavorm>adipositas</Lemmavorm> <Lemmatype>woord</Lemmatype> </Lemma> <SpellingDbId>101775</SpellingDbId> <Woordsoort> <Type>substantief</Type> <Substantief> <Naamtype>soortnaam</Naamtype> <GeslachtBody id="el6"> <Geslacht>vrouwelijk</Geslacht> </GeslachtBody> <Lidwoord id="el7"> <AardLidwoord>de</AardLidwoord> </Lidwoord> <Getal>geen meervoud</Getal> <Betekenisklasse>abstractum</Betekenisklasse> </Substantief> </Woordsoort> <SpellingEnFlexie> <SubstantiefVormen> <Enkelvoud LexiconId="109674"> <Woordvorm>adipositas</Woordvorm> <Afbreking>adi.po.si.tas</Afbreking> </Enkelvoud> </SubstantiefVormen> </SpellingEnFlexie> <Uitspraak> <AantalLettergrepenBody id="el8"> <AantalLettergrepen>5</AantalLettergrepen> </AantalLe

In [25]:
count = df[df['xml_file'].str.decode('utf-8').str.contains('Voorbeeld')].shape[0]
print(count)

28781


In [26]:
filtered_df = df[df['xml_file'].str.decode('utf-8').str.contains('Voorbeeld')]
print(filtered_df)

                        word  \
2                  06-dealer   
3                  06-nummer   
4             06-prostitutie   
7          1,5 metereconomie   
8      1,5 metermaatschappij   
...                      ...   
86574             Zwollenaar   
86576                 Zwolse   
86578                 zzp'en   
86579                 zzp'er   
86580                    ZZS   

                                                xml_file  
2      b'<artikel last-modified="2024-02-23" ID="7121...  
3      b'<artikel last-modified="2024-03-01" ID="6896...  
4      b'<artikel last-modified="2023-10-04" ID="8749...  
7      b'<artikel last-modified="2024-03-01" ID="1599...  
8      b'<artikel last-modified="2024-03-01" ID="1616...  
...                                                  ...  
86574  b'<artikel last-modified="2023-10-04" ID="6873...  
86576  b'<artikel last-modified="2023-10-04" ID="7075...  
86578  b'<artikel last-modified="2023-11-01" ID="1553...  
86579  b'<artikel last-mo

In [27]:
filtered_df.head()

,word,xml_file
2,06-dealer,"b'<artikel last-modified=""2024-02-23"" ID=""7121..."
3,06-nummer,"b'<artikel last-modified=""2024-03-01"" ID=""6896..."
4,06-prostitutie,"b'<artikel last-modified=""2023-10-04"" ID=""8749..."
7,"1,5 metereconomie","b'<artikel last-modified=""2024-03-01"" ID=""1599..."
8,"1,5 metermaatschappij","b'<artikel last-modified=""2024-03-01"" ID=""1616..."


In [28]:
filtered_df.iloc[0]['xml_file']

b'<artikel last-modified="2024-02-23" ID="71214" pid="190832"> <Lemma> <Lemmavorm>06-dealer</Lemmavorm> <Lemmatype>woord</Lemmatype> </Lemma>       <BetekenisEnGebruik> <Kernbetekenis pid="190834" id="bet1.0"> <betekenisInfo> <Betekenisnummer>1.0</Betekenisnummer> <Lemma> <Lemmavorm>06-dealer</Lemmavorm> <Lemmatype>woord</Lemmatype> </Lemma> <SpellingDbId>170109</SpellingDbId> <Woordsoort> <Type>substantief</Type> <Substantief> <Naamtype>soortnaam</Naamtype> <GeslachtBody id="el6"> <Geslacht>mannelijk</Geslacht> </GeslachtBody> <Lidwoord id="el7"> <AardLidwoord>de</AardLidwoord> </Lidwoord> <Betekenisklasse>persoonsnaam</Betekenisklasse> </Substantief> </Woordsoort> <SpellingEnFlexie> <SubstantiefVormen> <Enkelvoud LexiconId="504881"> <Woordvorm>06-dealer</Woordvorm> <Afbreking>06-.dea.ler</Afbreking> </Enkelvoud> <Meervoud LexiconId="470570"> <Woordvorm>06-dealers</Woordvorm> <Afbreking>06-.dea.lers</Afbreking> </Meervoud> <Verkleiningsvorm LexiconId="782607"> <Woordvorm>06-dealertje<

In [29]:
filtered_df.to_csv('Data/dut_general_example_filtered_xml.csv', index = False)